In [4]:
!pip install torch torchvision

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [5]:
import logging
import random

import numpy as np
import torch

logging.basicConfig(level=logging.INFO, format='%(asctime)-15s %(levelname)s: %(message)s')

# set seed
seed = 666
random.seed(seed)
np.random.seed(seed)
torch.cuda.manual_seed(seed)
torch.manual_seed(seed)

In [21]:
# split data to 10 fold
fold_num = 10
data_file = '../input/train_set.csv'
import pandas as pd


def all_data2fold(fold_num, num=10000):
    fold_data = []
    
    # 读取数据文件
    f = pd.read_csv(data_file, sep='\t', encoding='UTF-8')
    
    # 读取到num到的数据到text
    texts = f['text'].tolist()[:num]
    
    # 读取到num到的数据到label
    labels = f['label'].tolist()[:num]
    
    # 获取标签数
    total = len(labels)

    # 使用标签数，生成列表
    index = list(range(total))
    
    # 打乱顺序
    np.random.shuffle(index)

    # 定义所有text
    all_texts = []
    # 定义所有label
    all_labels = []
    
    for i in index:
        # 把打乱顺序的text，放进所有text
        all_texts.append(texts[i])
        
        # 把打乱顺序的label，放进所有label
        all_labels.append(labels[i])

    label2id = {}
    
    #给每个label赋予索引值
    for i in range(total):
        
        # 按索引顺序 ，从打乱顺序的所有label例取出label
        label = str(all_labels[i])
        
        # 如果 label不在label2id里的话：
        if label not in label2id:
            # 给这个label更新索引值
            label2id[label] = [i]
        else:
            # 给这个label追加索引值
            label2id[label].append(i)
    
    # 定义fold_num个列表
    all_index = [[] for _ in range(fold_num)]
    
    # 遍历label2id里每个label和data
    for label, data in label2id.items():
        # print(label, len(data))
        
        # data和fold_num来确定批量尺寸
        batch_size = int(len(data) / fold_num)
        
        # 
        other = len(data) - batch_size * fold_num
        for i in range(fold_num):
            cur_batch_size = batch_size + 1 if i < other else batch_size
            # print(cur_batch_size)
            batch_data = [data[i * batch_size + b] for b in range(cur_batch_size)]
            all_index[i].extend(batch_data)

    batch_size = int(total / fold_num)
    other_texts = []
    other_labels = []
    other_num = 0
    start = 0
    for fold in range(fold_num):
        num = len(all_index[fold])
        texts = [all_texts[i] for i in all_index[fold]]
        labels = [all_labels[i] for i in all_index[fold]]

        if num > batch_size:
            fold_texts = texts[:batch_size]
            other_texts.extend(texts[batch_size:])
            fold_labels = labels[:batch_size]
            other_labels.extend(labels[batch_size:])
            other_num += num - batch_size
        elif num < batch_size:
            end = start + batch_size - num
            fold_texts = texts + other_texts[start: end]
            fold_labels = labels + other_labels[start: end]
            start = end
        else:
            fold_texts = texts
            fold_labels = labels

        assert batch_size == len(fold_labels)

        # shuffle
        index = list(range(batch_size))
        np.random.shuffle(index)

        shuffle_fold_texts = []
        shuffle_fold_labels = []
        for i in index:
            shuffle_fold_texts.append(fold_texts[i])
            shuffle_fold_labels.append(fold_labels[i])

        data = {'label': shuffle_fold_labels, 'text': shuffle_fold_texts}
        fold_data.append(data)

    logging.info("Fold lens %s", str([len(data['label']) for data in fold_data]))

    return fold_data


fold_data = all_data2fold(10)

2020-07-27 16:48:09,368 INFO: Fold lens [1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000, 1000]


In [22]:
# build train data for word2vec
fold_id = 9

train_texts = []
for i in range(0, fold_id):
    data = fold_data[i]
    train_texts.extend(data['text'])
    
logging.info('Total %d docs.' % len(train_texts))

2020-07-27 16:48:20,829 INFO: Total 9000 docs.


In [24]:
!pip install -U gensim

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Requirement already up-to-date: gensim in /Users/admin/anaconda3/lib/python3.7/site-packages (3.8.3)


In [25]:
logging.info('Start training...')
from gensim.models.word2vec import Word2Vec

num_features = 100     # Word vector dimensionality
num_workers = 8       # Number of threads to run in parallel

train_texts = list(map(lambda x: list(x.split()), train_texts))
model = Word2Vec(train_texts, workers=num_workers, size=num_features)
model.init_sims(replace=True)

# save model
model.save("./word2vec.bin")

2020-07-27 16:56:54,722 INFO: Start training...
2020-07-27 16:56:55,965 INFO: collecting all words and their counts
2020-07-27 16:56:55,966 INFO: PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-07-27 16:56:57,575 INFO: collected 5267 word types from a corpus of 8162291 raw words and 9000 sentences
2020-07-27 16:56:57,576 INFO: Loading a fresh vocabulary
2020-07-27 16:56:57,628 INFO: effective_min_count=5 retains 4334 unique words (82% of original 5267, drops 933)
2020-07-27 16:56:57,628 INFO: effective_min_count=5 leaves 8160401 word corpus (99% of original 8162291, drops 1890)
2020-07-27 16:56:57,663 INFO: deleting the raw counts dictionary of 5267 items
2020-07-27 16:56:57,664 INFO: sample=0.001 downsamples 61 most-common words
2020-07-27 16:56:57,666 INFO: downsampling leaves estimated 7044847 word corpus (86.3% of prior 8160401)
2020-07-27 16:56:57,690 INFO: estimated required memory for 4334 words and 100 dimensions: 5634200 bytes
2020-07-27 16:56:57,693 INF

2020-07-27 16:57:35,915 INFO: EPOCH 5 - PROGRESS: at 91.04% examples, 905856 words/s, in_qsize 16, out_qsize 0
2020-07-27 16:57:36,560 INFO: worker thread finished; awaiting finish of 7 more threads
2020-07-27 16:57:36,570 INFO: worker thread finished; awaiting finish of 6 more threads
2020-07-27 16:57:36,571 INFO: worker thread finished; awaiting finish of 5 more threads
2020-07-27 16:57:36,578 INFO: worker thread finished; awaiting finish of 4 more threads
2020-07-27 16:57:36,598 INFO: worker thread finished; awaiting finish of 3 more threads
2020-07-27 16:57:36,608 INFO: worker thread finished; awaiting finish of 2 more threads
2020-07-27 16:57:36,620 INFO: worker thread finished; awaiting finish of 1 more threads
2020-07-27 16:57:36,634 INFO: worker thread finished; awaiting finish of 0 more threads
2020-07-27 16:57:36,635 INFO: EPOCH - 5 : training on 8162291 raw words (7008674 effective words) took 7.8s, 900692 effective words/s
2020-07-27 16:57:36,638 INFO: training on a 4081145

In [26]:
# load model
model = Word2Vec.load("./word2vec.bin")

# convert format
model.wv.save_word2vec_format('./word2vec.txt', binary=False)

2020-07-27 16:58:13,179 INFO: loading Word2Vec object from ./word2vec.bin
2020-07-27 16:58:13,299 INFO: loading wv recursively from ./word2vec.bin.wv.* with mmap=None
2020-07-27 16:58:13,300 INFO: setting ignored attribute vectors_norm to None
2020-07-27 16:58:13,301 INFO: loading vocabulary recursively from ./word2vec.bin.vocabulary.* with mmap=None
2020-07-27 16:58:13,302 INFO: loading trainables recursively from ./word2vec.bin.trainables.* with mmap=None
2020-07-27 16:58:13,303 INFO: setting ignored attribute cum_table to None
2020-07-27 16:58:13,304 INFO: loaded ./word2vec.bin
2020-07-27 16:58:13,323 INFO: storing 4334x100 projection weights into ./word2vec.txt


In [29]:
# 读取测试集数据
test_df = pd.read_csv('../input/test_a.csv', sep='\t')
